<center><img src='https://drive.google.com/uc?id=1_utx_ZGclmCwNttSe40kYA6VHzNocdET' height="60"></center>

AI TECH - Akademia Innowacyjnych Zastosowań Technologii Cyfrowych. Program Operacyjny Polska Cyfrowa na lata 2014-2020
<hr>

<center><img src='https://drive.google.com/uc?id=1BXZ0u3562N_MqCLcekI-Ens77Kk4LpPm'></center>

<center>
Projekt współfinansowany ze środków Unii Europejskiej w ramach Europejskiego Funduszu Rozwoju Regionalnego
Program Operacyjny Polska Cyfrowa na lata 2014-2020,
Oś Priorytetowa nr 3 "Cyfrowe kompetencje społeczeństwa" Działanie  nr 3.2 "Innowacyjne rozwiązania na rzecz aktywizacji cyfrowej"
Tytuł projektu:  „Akademia Innowacyjnych Zastosowań Technologii Cyfrowych (AI Tech)”
    </center>

# Validation and cross-validation

In this exercise you will implement a validation pipeline.

At the end of the MSLE exercise you tested your model against the training and test datasets. As you should observe, there's a gap between the results. By validating your model, not only should you be able to anticipate the test time performance, but also have a method to compare different models.

Implement the basic validation method, i.e. a random split. Test it with your model from Exercise MSLE.

In [1]:
%matplotlib inline

!wget -O mieszkania.csv https://www.dropbox.com/s/zey0gx91pna8irj/mieszkania.csv?dl=1
!wget -O mieszkania_test.csv https://www.dropbox.com/s/dbrj6sbxb4ayqjz/mieszkania_test.csv?dl=1

--2023-10-30 09:50:46--  https://www.dropbox.com/s/zey0gx91pna8irj/mieszkania.csv?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.6.18, 2620:100:601c:18::a27d:612
Connecting to www.dropbox.com (www.dropbox.com)|162.125.6.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/dl/zey0gx91pna8irj/mieszkania.csv [following]
--2023-10-30 09:50:46--  https://www.dropbox.com/s/dl/zey0gx91pna8irj/mieszkania.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc434beadad906a0d82c49ee6fca.dl.dropboxusercontent.com/cd/0/get/CGny4RerxwL8KnKzd8t2okO2WnBSbRCJkURxt_7ji6_xEmPJcLVJQAGDgZDMjOPFaqogtfTGPMqZfxM3BUujc-tG4W15LvN-pUar0cY-cFpGtDnfhHTI4H9DoO_d_JahbgE/file?dl=1# [following]
--2023-10-30 09:50:47--  https://uc434beadad906a0d82c49ee6fca.dl.dropboxusercontent.com/cd/0/get/CGny4RerxwL8KnKzd8t2okO2WnBSbRCJkURxt_7ji6_xEmPJcLVJQAGDgZDMjOPFaqogtfTGPMqZfxM3BUujc-tG4W15LvN-pUar0cY-cFpGtDnfhHT

In [2]:
import numpy as np
import pandas as pd

def load(name: str) -> tuple[np.ndarray, np.array]:
    data = pd.read_csv(name)
    x = data.loc[:, data.columns != 'cena'].to_numpy()
    y = data['cena'].to_numpy()

    return x, y

In [3]:
x_train, y_train = load('mieszkania.csv')
x_test, y_test = load('mieszkania_test.csv')

In [4]:
from sklearn import preprocessing

labelencoder = preprocessing.LabelEncoder()
labelencoder.fit(x_train[:, 1])
x_train[:, 1] = labelencoder.transform(x_train[:, 1])
x_test[:, 1] = labelencoder.transform(x_test[:, 1])

x_train = x_train.astype(np.float64)
x_test = x_test.astype(np.float64)

In [6]:
#############################################
# TODO: Optimize msle and compare the error #
#############################################
def msle(y_true, y_pred):
  return np.mean((np.log(1+y_true)-np.log(1+y_pred))**2)

def weights_gradient(weights, bias, X, y):
  y_pred = np.einsum('ij,j->i', X, weights)+bias
  return -2/len(X) * np.matmul((np.log(1+y) - np.log(1+y_pred)), X)

def bias_gradient(weights, bias, X, y):
  y_pred = np.einsum('ij,j->i', X, weights)+bias
  return -2/len(X) * np.mean(np.log(1+y) - np.log(1+y_pred))

def gradients(weights, bias, X, y):
  return weights_gradient(weights, bias, X, y), bias_gradient(weights, bias, X, y)

num_epochs = 500
lr = 0.1

def train(num_epochs, lr, X, y, print_logs = True):

  weights = np.zeros(shape=len(X[0]))
  bias = 0

  for i in range(num_epochs):
    g_w, g_b = gradients(weights, bias, X, y)
    weights -= lr*g_w
    bias -= lr*g_b
    p = np.matmul(X, weights)+bias
    loss = msle(y, p)

    if print_logs:
      print(f'Iter: {i:3} Loss: {loss:2.3f} Weight 0: {weights[0]:8.5f}, Bias: {bias:8.5f}')

  return weights, bias

train(num_epochs, lr, x_train, y_train)

Iter:   0 Loss: 10.243 Weight 0: 211.97681, Bias:  0.01297
Iter:   1 Loss: 8.577 Weight 0: 165.77836, Bias:  0.00983
Iter:   2 Loss: 6.722 Weight 0: 124.01937, Bias:  0.00697
Iter:   3 Loss: 4.660 Weight 0: 87.77349, Bias:  0.00444
Iter:   4 Loss: 2.454 Weight 0: 58.71330, Bias:  0.00237
Iter:   5 Loss: 0.622 Weight 0: 39.68278, Bias:  0.00092
Iter:   6 Loss: 0.435 Weight 0: 35.62311, Bias:  0.00041
Iter:   7 Loss: 0.411 Weight 0: 44.20116, Bias:  0.00069
Iter:   8 Loss: 0.383 Weight 0: 44.66249, Bias:  0.00046
Iter:   9 Loss: 0.375 Weight 0: 51.35748, Bias:  0.00062
Iter:  10 Loss: 0.366 Weight 0: 53.35367, Bias:  0.00049
Iter:  11 Loss: 0.363 Weight 0: 59.01405, Bias:  0.00058
Iter:  12 Loss: 0.359 Weight 0: 61.84398, Bias:  0.00050
Iter:  13 Loss: 0.358 Weight 0: 66.88795, Bias:  0.00056
Iter:  14 Loss: 0.356 Weight 0: 70.20535, Bias:  0.00051
Iter:  15 Loss: 0.356 Weight 0: 74.87234, Bias:  0.00055
Iter:  16 Loss: 0.355 Weight 0: 78.48120, Bias:  0.00052
Iter:  17 Loss: 0.355 Weigh

(array([1.75861739e+03, 1.10730343e+01, 3.26113977e+01, 2.36625923e+01,
        1.51125568e+02, 7.20532474e-01]),
 -0.00022924557938148213)

In [7]:
def test(weights, bias, X, y):

  p = np.matmul(X, weights)+bias
  loss = msle(y, p)

  return loss

In [8]:
#######################################################
# TODO: Implement the basic validation method,        #
# compare MSLE on training, validation, and test sets #
#######################################################
def msle(ys, ps):
    assert len(ys) == len(ps)
    return np.mean((np.log(1+np.array(ys))-np.log(1+np.array(ps)))**2)

def msle_loss(weights, bias, X, y):
    p = np.matmul(X, weights)+bias
    loss = msle(y, p)
    return loss

def msle_train_val(weights, bias, X_train, y_train, X_val, y_val):
  return msle_loss(weights, bias, X_train, y_train), msle_loss(weights, bias, X_val, y_val)

To make the random split validation reliable, a huge chunk of training data may be needed. To get over this problem, one may apply cross-validaiton.

![alt-text](https://chrisjmccormick.files.wordpress.com/2013/07/10_fold_cv.png)

Let's now implement the method. Make sure that:
* number of partitions is a parameter,
* the method is not limited to `mieszkania.csv`,
* the method is not limited to one specific model.

In [9]:
from os import terminal_size
####################################
# TODO: Implement cross-validation #
####################################
from sklearn.model_selection import KFold

def cross_validation(num_part, x_train, y_train):

  num_epochs = 100
  lr = 0.01
  kf = KFold(n_splits=num_part, shuffle=True)
  train_losses = []
  val_losses = []

  for i, (train_index, val_index) in enumerate(kf.split(x_train)):
    #  print(f"Fold {i}:")
    #  print(f"  Train: index={train_index}")
    #  print(f"  Test:  index={val_index}")

     x_t = x_train[train_index]
     y_t = y_train[train_index]
     x_v = x_train[val_index]
     y_v = y_train[val_index]

     weights, bias = train(num_epochs, lr, x_t, y_t, print_logs=False)

     t_l, v_l = msle_train_val(weights, bias, x_t, y_t, x_v, y_v)

     train_losses.append(t_l)
     val_losses.append(v_l)

     print(f"Fold {i}:")
     print(f"  Train loss={t_l}")
     print(f"  Val   loss={v_l}")

  return np.mean(train_losses), np.mean(val_losses)

cross_validation(10, x_train, y_train)

Fold 0:
  Train loss=0.33627142381452024
  Val   loss=0.5536976253582033
Fold 1:
  Train loss=0.35679198338211127
  Val   loss=0.3602080580048924
Fold 2:
  Train loss=0.36513703462373265
  Val   loss=0.2849131800205593
Fold 3:
  Train loss=0.3524465543458869
  Val   loss=0.39951603109745387
Fold 4:
  Train loss=0.3639115496362545
  Val   loss=0.2960780340294005
Fold 5:
  Train loss=0.3630682691353477
  Val   loss=0.30381432524819607
Fold 6:
  Train loss=0.35911356814733464
  Val   loss=0.3399325509333434
Fold 7:
  Train loss=0.35884353538519415
  Val   loss=0.342292284364446
Fold 8:
  Train loss=0.3601078181314308
  Val   loss=0.32983006003362425
Fold 9:
  Train loss=0.3544794316138101
  Val   loss=0.3849808582620006


(0.35701711682156223, 0.35952630073521197)

Recall that sometimes validation may be tricky, e.g. significant class imbalance, having a small number of subjects, geographically clustered instances...

What could in theory go wrong here with random, unstratified partitions? Think about potential solutions and investigate the data in order to check whether these problems arise here.

In [11]:
##############################
# TODO: Investigate the data #
##############################
data = pd.read_csv('mieszkania.csv')
labelencoder = preprocessing.LabelEncoder()
data['dzielnica'] = labelencoder.fit_transform(data['dzielnica'])
data.head()

,m2,dzielnica,ilość_sypialni,ilość_łazienek,rok_budowy,parking_podziemny,cena
0,104,1,2,2,1940,1,780094
1,43,2,1,1,1970,1,346912
2,128,0,3,2,1916,1,523466
3,112,1,3,2,1920,1,830965
4,149,1,3,3,1977,0,1090479


In [13]:
import plotly.express as px

for column in data.columns:
  fig = px.histogram(data, x=column)
  fig.show()

In [ ]:
# Conclusions:
# - little data about flats 1.1mln+, built after 2010, with 3 bathrooms

In [14]:
for column in data.columns:
  fig = px.scatter(data, x=column, y="cena")
  fig.show()

In [16]:
px.scatter_3d(data, x="m2", y="rok_budowy", z="cena")